In [1]:
# %%
import time
import numpy as np
import pickle
from numpy.linalg import det

import CMINE_lib as CMINE
# from Guassian_variables import Data_guassian
from structurerl import * 

import pandas as pd
from scipy.stats import multivariate_normal
import itertools

np.random.seed(37)
from scipy import stats
from sklearn.neighbors import KernelDensity

import math

import torch 
import torch.nn as nn
import torch.nn.functional as F

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'


In [2]:
def log_sum_exp(value, dim=None, keepdim=False):
    """Numerically stable implementation of the operation
    value.exp().sum(dim, keepdim).log()
    """
    # TODO: torch.max(value, dim=None) threw an error at time of writing
    if dim is not None:
        m, _ = torch.max(value, dim=dim, keepdim=True)
        value0 = value - m
        if keepdim is False:
            m = m.squeeze(dim)
        return m + torch.log(torch.sum(torch.exp(value0),
                                       dim=dim, keepdim=keepdim))
    else:
        m = torch.max(value)
        sum_exp = torch.sum(torch.exp(value - m))
        if isinstance(sum_exp, Number):
            return m + math.log(sum_exp)
        else:
            return m + torch.log(sum_exp)

In [3]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

In [4]:
from modules.CMI import DR_CMI, CDL_CMI

In [5]:
Dim = 5
batch_size = 64
#dataset = CMINE.create_dataset_DGP( Dim=5, N=batch_size)
dataset = create_dataset_DGP_binary_A_conf( Dim=5, N=batch_size)
s_t = torch.from_numpy(dataset[0]).float().cuda()
s_next = torch.from_numpy(dataset[1]).float().cuda()
a = torch.from_numpy(dataset[2]).float().cuda()

In [6]:
sample_dim = 2*Dim

hidden_size = 15
learning_rate = 0.005
training_steps = 10

cubic = False 


In [7]:
def train_dr(N = 64, training_steps = 10):
    torch.cuda.empty_cache()
    model_dr = DR_CMI(sample_dim + 1, sample_dim, hidden_size).cuda()
    optimizer_dr = torch.optim.Adam(model_dr.parameters(), learning_rate)
    dr_est_values = []
    for step in range(training_steps):
        #batch_x, batch_y = sample_correlated_gaussian(rho, dim=sample_dim, batch_size = batch_size, to_cuda = True, cubic = cubic)
        dataset = create_dataset_DGP_binary_A_conf(Dim=Dim, N=N)
        #dataset = create_dataset_DGP_binary_A(Dim=Dim, N=N)
        s_t = torch.from_numpy(dataset[0]).float().cuda()
        s_next = torch.from_numpy(dataset[1]).float().cuda()
        a = torch.from_numpy(dataset[2]).float().cuda()
        #print(a)

        batch_x = torch.cat([s_t,a], dim=1)
        batch_y = s_next
        model_dr.eval()
        drs = model_dr(batch_x, batch_y)
        #mi_est_values.append(cmi)
        dr_est_values.append(drs)
        model_dr.train() 

        model_loss = model_dr.learning_loss(batch_x, batch_y)

        optimizer_dr.zero_grad()
        model_loss.backward(retain_graph=True)
        optimizer_dr.step()

        del batch_x, batch_y
        torch.cuda.empty_cache()
    return dr_est_values

In [8]:
def train_cdl(N = 64, training_steps = 10):
    torch.cuda.empty_cache()
    model_cdl = CDL_CMI(sample_dim + 1, sample_dim, hidden_size).cuda()
    optimizer_cdl = torch.optim.Adam(model_cdl.parameters(), learning_rate)
    cdl_est_values = []
    for step in range(training_steps):
        #batch_x, batch_y = sample_correlated_gaussian(rho, dim=sample_dim, batch_size = batch_size, to_cuda = True, cubic = cubic)
        dataset = create_dataset_DGP_binary_A_conf(Dim=Dim, N=64)
        #dataset = create_dataset_DGP_binary_A(Dim=Dim, N=64)
        s_t = torch.from_numpy(dataset[0]).float().cuda()
        s_next = torch.from_numpy(dataset[1]).float().cuda()
        a = torch.from_numpy(dataset[2]).float().cuda()

        batch_x = torch.cat([s_t,a], dim=1)
        batch_y = s_next
        model_cdl.eval()
        cdl_cmi = model_cdl(batch_x, batch_y)
        cdl_est_values.append(cdl_cmi)
        model_cdl.train() 

        model_loss = model_cdl.learning_loss(batch_x, batch_y)

        optimizer_cdl.zero_grad()
        model_loss.backward(retain_graph=True)
        optimizer_cdl.step()

        del batch_x, batch_y
        torch.cuda.empty_cache()
    return cdl_est_values

In [9]:
N=64
cdl_est_values = train_cdl(N, 10)
dr_est_values = train_dr(N, 10)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[1.56100398e-01 6.17612654e-01 2.01037466e-01 1.47533782e-01
 3.83129553e+00 1.58807712e+02 5.13441795e+01 4.92158972e+01
 1.03007042e+06 1.55195305e+03]
[1.77655655e+04 1.84648281e+04 1.78624945e+04 1.77955943e+04
 1.79831217e+04 3.45603236e+06 2.86323051e+08 6.54707734e+11
 2.94978583e+09 2.01939495e+08]


In [10]:
N=128
cdl_est_values = train_cdl(N, 10)
dr_est_values = train_dr(N, 10)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[1.52500487e+00 3.68097122e-01 3.33922929e-01 4.01493548e+00
 2.50107577e-01 8.83974360e+04 1.22496091e+06 3.70694904e+03
 1.78455402e+02 1.28071267e+04]
[1.56739116e+01 4.01772423e+02 1.40300239e+01 3.09470231e+01
 4.81554096e+00 2.05146616e+05 6.19165898e+00 4.29090376e+05
 1.42412817e+03 6.02665755e+05]


In [12]:
N = 32
cdl_est_values = train_cdl(N, 10)
dr_est_values = train_dr(N, 10)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[1.51777699e+00 1.23980819e+00 1.81605854e+00 1.04856310e+00
 3.52862078e+03 3.50230264e+03 3.09710035e+04 8.14561931e+03
 6.34948409e+02 1.31929998e+05]
[3.47660793e+02 9.71036765e+01 2.75425163e+00 2.21595057e+02
 2.60199978e+01 9.53826048e+05 1.82733502e+09 1.52683709e+05
 1.98837776e+05 3.51046172e+06]


In [13]:
N = 16
cdl_est_values = train_cdl(N, 10)
dr_est_values = train_dr(N, 10)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[0.83482626 0.07871534 0.05686883 4.96503454 0.04981554 0.47070297
 0.20962007 1.00761549 0.23704674 1.41036557]
[2.45487296e+02 2.07110710e+02 2.78355743e+02 2.70922651e+02
 3.21791865e+02 1.06133390e+05 2.84176001e+04 1.05634527e+04
 1.17334941e+08 2.06357747e+04]


In [14]:
N = 8
cdl_est_values = train_cdl(N, 10)
dr_est_values = train_dr(N, 10)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[3.64212415e+02 3.63155174e+02 3.62741391e+02 3.63982946e+02
 3.62547022e+02 3.54660821e+06 1.17143885e+05 6.30458812e+03
 2.02804373e+05 6.58121819e+03]
[5.34660669e+06 5.39030692e+06 5.34102615e+06 5.31054044e+06
 5.68449332e+06 5.41413441e+16 8.78900676e+08 1.34109684e+10
 1.45785828e+12 2.63270089e+10]


In [15]:
N = 8
cdl_est_values = train_cdl(N, 100)
dr_est_values = train_dr(N, 100)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[2.22504738e+01 2.67431462e+06 7.73034164e+02 3.96541044e+02
 1.43597268e+02 4.97873293e+04 2.88397809e+03 5.05354645e+05
 7.70204409e+02 7.59108282e+03]
[2.53562779e+04 6.14453626e+06 1.12875001e+05 2.62410940e+03
 1.53140977e+03 3.24888070e+04 4.25532312e+04 8.11987926e+03
 1.10298786e+08 2.56945881e+04]


# Training Step

In [26]:
N = 64
training_step = 100
cdl_est_values = train_cdl(N, training_step)
dr_est_values = train_dr(N, training_step)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[6.31943605e+04 1.71830261e+08 6.60621143e+04 1.83854909e+05
 6.29557521e+04 3.95083027e+09 5.30933075e+05 3.03847897e+05
 3.01877772e+06 9.41156254e+05]
[1.45674717e+05 5.62305784e+05 1.31564423e+05 1.34648689e+05
 1.32427251e+06 2.39606343e+09 1.46911331e+14 6.66713691e+09
 1.28531635e+08 2.40902858e+09]


In [27]:
N = 32
training_step = 100
cdl_est_values = train_cdl(N, training_step)
dr_est_values = train_dr(N, training_step)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[1.68653295e+04 8.02705649e+00 1.19907736e+03 3.14780780e+01
 3.98932782e+01 8.38730004e+05 3.92318122e+03 6.95953489e+04
 1.22019822e+05 3.48546747e+06]
[9.74688966e+03 7.74597234e+03 2.96815940e+09 6.31418542e+03
 6.20089638e+03 1.22671743e+06 3.68697284e+09 4.77083438e+06
 1.44713419e+07 1.88002237e+07]


In [31]:
N = 526
training_step = 100
cdl_est_values = train_cdl(N, training_step)
dr_est_values = train_dr(N, training_step)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[1.50006065e+01 2.18871596e+02 1.10105989e+07 5.11114454e+02
 1.06051950e+02 9.00031509e+01 1.04145438e+04 3.52512225e+03
 5.52125692e+03 2.41971664e+04]
[7.68950225e+06 7.74432824e+06 7.91731634e+06 7.70377130e+06
 7.70934032e+06 7.88427070e+08 4.68894041e+16 1.52884448e+10
 1.34091251e+11 2.40433729e+10]


In [30]:
N = 526
training_step = 10
cdl_est_values = train_cdl(N, training_step)
dr_est_values = train_dr(N, training_step)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[2.41623130e-01 5.83663910e-01 2.62432172e-01 1.86141613e-01
 2.88988191e+02 2.00060656e+01 5.57397880e+00 2.69901884e+01
 2.13138771e-01 1.53580540e+00]
[1.34774063e+06 7.49846064e+03 7.87508115e+03 7.57209025e+03
 7.50388969e+03 5.57463735e+05 1.85607135e+04 3.51530271e+09
 1.84042864e+05 2.73516473e+06]
